In [1]:
import socket
import ssl
import os
import tkinter as tk
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


In [2]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

# def derive_dh_shared_key(private_key, public_key):
#     shared_key = private_key.exchange(public_key)
#     return shared_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = public_key.public_numbers().y ** private_key.private_numbers().x % public_key.public_numbers().parameter_numbers.g
    return shared_key.to_bytes((shared_key.bit_length() + 7) // 8, byteorder='big')


def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 8443))
    server_socket.listen(1)

    print("Server listening on port 8443...")

    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address} has been established!")

        context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
        print("SSL/TLS context created.")
        context.load_cert_chain(certfile="server.crt", keyfile="server.key")
        print("SSL/TLS context loaded.")
        
        with context.wrap_socket(client_socket, server_side=True) as ssock:
            print("SSL/TLS connection established.")
            encryption_choice = ssock.recv(1024).decode()
            print(f"Client chose encryption method {encryption_choice}")
            
            if encryption_choice == "1":
                pb_key = ssock.recv(1024)
                print(f"Key received from client: {pb_key.decode()}") 
                
                private_key = generate_rsa_keypair()
                public_key = private_key.public_key()
                
                #send public key to client
                ssock.send(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                ))
 
                client_public_key = load_pem_public_key(pb_key, backend=default_backend())
                print(f"Client public key: {client_public_key}")
                 
                while True:
                    msg_from_client = ssock.recv(1024)
                    print(f"Encrypted Message from client: {msg_from_client}")
                    msg_from_client = rsa_decrypt(private_key, msg_from_client)
                    print(f"Decrypted message from client: {msg_from_client}")
                    if msg_from_client == "exit":
                        break
                    
                    msg_for_client = input("Enter message for client: ")
                    msg_for_client = rsa_encrypt(client_public_key, msg_for_client)
                    ssock.send(msg_for_client)

            elif encryption_choice == "2":
                # AES encryption and Diffie-Hellman key exchange
                received_public_key = ssock.recv(2048)
                print(f"Received public key: {received_public_key}")
                received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
                
                private_key, public_key = generate_dh_keypair()
                
                ssock.send(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                ))
                print(f"Public key sent: {public_key}")
                print(f"Private key: {private_key}")
                shared_key = derive_dh_shared_key(private_key, received_public_key)
                print(f"Shared key: {shared_key}")
                while True:
                    msg_from_client = ssock.recv(2048)
                    msg_from_client = aes_decrypt(shared_key, msg_from_client)
                    print(f"Message from client: {msg_from_client}")
                    if msg_from_client == "exit":
                        break
                    msg_for_client = input("Enter message for client: ")
                    msg_for_client = aes_encrypt(shared_key, msg_for_client)
                    ssock.send(msg_for_client)




In [3]:
if __name__ == "__main__":
    start_server()


Server listening on port 8443...
Connection from ('127.0.0.1', 61965) has been established!
SSL/TLS context created.
SSL/TLS context loaded.
SSL/TLS connection established.
Client chose encryption method 1
Key received from client: -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAszjVqZjSDGErPuEEGse/
ISSHcbAF6dLFQxqc9zD6+y6Mh4tyBivoq5z5rM2X4pU5mETQuot3GKCwlsKEwEPI
ySogmv9Ktsr1hIUEemHryXPKjNhNWqXrdF4M8wkaVVw7KD70yzoIMNzyW+s7MRDc
Gn5PhemA/eY0rKg0woqm4wtfjtf52lhnTQqwVGMTxr7gvmioC8hTm82MC+37Ed2J
coMWvdE4RjzuCmLeAQnB51y6y39gpHbBb62g0ROhfsvYOZocX6aK4GR0rVDUseBZ
XFuAq/uu2L+EHtQWBpzIoNOyd6YZW72shzYa0J3u4Ma+7YWw45YawLfJuxob2WFh
AwIDAQAB
-----END PUBLIC KEY-----

Client public key: <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x000001A220E72500>
Encrypted Message from client: b"\xb1[8\x8e.T\xc3\xb0\xe7\xa5\x92q<\x00%d\x05$\x80'\xe7_\xd2\xd9{\xe4i\xa2\xe7\xd2\x03\x88\xc4\xc6%\xf0\xe3\x1f\x06\xdfC\x17L\xb4\xe6\xc5\x1eB\x88\x07\xa1\xe5\xf4\xaa\x0b%GD\x10\x

In [ ]:
# # Generate a private key for the server
# openssl genpkey -algorithm RSA -out server.key

# # Create a Certificate Signing Request (CSR) for the server
# openssl req -new -key server.key -out server.csr

# # Sign the server CSR with the CA certificate
# openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out server.crt 